## Plan simplifié pour diagonaliser une matrice $A$

1. **Vérifier que $A$ est carrée**  
   - Sinon, pas de diagonalisation possible.

2. **Calculer $\det(A)$**  
   - Si $\det(A) \neq 0$, $A$ est inversible (mais pas forcément diagonalisable).  
   - Si $\det(A) = 0$, $A$ peut encore être diagonalisable, mais aura au moins une valeur propre nulle.

3. **Trouver les valeurs propres**  
   - Résoudre $\det(A - \lambda I) = 0$  
   - Obtenir les racines $\lambda_1, \dots, \lambda_m$ (certaines peuvent se répéter).

4. **Trouver les vecteurs propres**  
   - Pour chaque $\lambda_i$ : résoudre $(A - \lambda_i I) v = 0$  
   - Trouver une base de l’espace propre associé à $\lambda_i$.

5. **Vérifier l’indépendance**  
   - On doit obtenir **exactement $n$ vecteurs propres linéairement indépendants**.  
   - En pratique : former la matrice $P$ avec ces vecteurs en colonnes et vérifier que $\det(P) \neq 0$.

6. **Construire $P$ et $D$**  
   - $P$ = matrice dont les colonnes sont les vecteurs propres (dans le même ordre que les valeurs propres).  
   - $D$ = matrice diagonale $\mathrm{diag}(\lambda_1, \dots, \lambda_n)$.

7. **Vérifier la diagonalisation**  
   - Vérifier que $A P = P D$  
   - Équivalent : $P^{-1} A P = D$.


In [2]:
import numpy as np

np.set_printoptions(precision=6, suppress=True)

def is_symmetric(A, tol=1e-10):
    return np.allclose(A, A.T, atol=tol)

def matrix_rank(A, tol=1e-10):
    s = np.linalg.svd(A, compute_uv=False)
    if s.size == 0:
        return 0
    return int(np.sum(s > tol * max(A.shape) * (s[0] if s[0] != 0 else 1)))

def nullspace(A, tol=1e-10):
    """
    Base (colonnes) du noyau de A via SVD.
    Retourne un array de taille (n, k) où k = dim(kernel).
    """
    U, S, Vh = np.linalg.svd(A)
    # Seuil relatif classique
    if S.size == 0:
        # Matrice nulle
        return np.eye(A.shape[1])
    thresh = tol * max(A.shape) * (S[0] if S[0] != 0 else 1)
    null_mask = (S <= thresh)
    # Colonnes de V correspondant aux petites valeurs singulières
    # Vh a shape (m, m); on prend les lignes où S ~ 0 (en queue)
    # Pour robuste : recompose à partir de Vh
    V = Vh.conj().T
    # Pour SVD numpy, les colonnes associées au noyau sont celles après le rang
    r = np.sum(S > thresh)
    return V[:, r:]

def cluster_eigenvalues(lambdas, tol=1e-8):
    """
    Regroupe les valeurs propres proches (permet de définir des 'lambda uniques'
    même en présence de bruit numérique).
    Retourne une liste de 'représentants' et une liste d'indices par cluster.
    """
    reps = []
    groups = []
    for i, lam in enumerate(lambdas):
        placed = False
        for g_idx, rep in enumerate(reps):
            if abs(lam - rep) <= tol:
                groups[g_idx].append(i)
                # Mise à jour du représentant (moyenne simple)
                reps[g_idx] = (reps[g_idx] * (len(groups[g_idx]) - 1) + lam) / len(groups[g_idx])
                placed = True
                break
        if not placed:
            reps.append(lam)
            groups.append([i])
    return np.array(reps, dtype=complex), groups

def diagonalize(A, tol=1e-10, verbose=True):
    """
    Tente de diagonaliser A.
    - Si A ~ symétrique => utilise eigh (réel et stable).
    - Sinon, construit des bases de noyaux pour chaque lambda et vérifie l'indépendance.

    Retour:
        success (bool), P (ou None), D (ou None), info (message)
    """
    A = np.array(A, dtype=complex)  # autorise le complexe
    n, m = A.shape
    if n != m:
        return False, None, None, "A n'est pas carrée."

    if is_symmetric(A.real, tol) and np.allclose(A.imag, 0, atol=tol):
        # cas symétrique réel: orthogonalement diagonalisable
        vals, vecs = np.linalg.eigh(A.real)
        P = vecs.astype(complex)
        D = np.diag(vals.astype(complex))
        if verbose:
            print("[Symétrique] Diagonalisation orthogonale trouvée.")
            print("P (orthonormale) =\n", P)
            print("D =\n", D)
            print("Vérif A P ~ P D :", np.allclose(A @ P, P @ D, atol=1e-8))
        return True, P, D, "Symétrique réelle: diagonalisation orthogonale."

    # Général : eig
    vals, vecs_eig = np.linalg.eig(A)
    # On clusterise pour stabiliser les lambdas égaux numériquement
    reps, groups = cluster_eigenvalues(vals, tol=1e-8)

    # Pour chaque lambda représentatif, on reconstruit une base du noyau(A - lambda I).
    eigenvectors = []
    diag_entries = []
    for rep, idxs in zip(reps, groups):
        # Base du noyau (A - rep I)
        K = nullspace(A - rep * np.eye(n), tol=tol)
        k = K.shape[1]
        if k == 0:
            # numériquement suspect; on tente au moins de prendre les vecteurs renvoyés par eig
            # pour ce cluster
            Vc = vecs_eig[:, idxs]
            # On orthonormalise ou réduit la dépendance via SVD
            # (on garde les colonnes indépendantes)
            r = matrix_rank(Vc, tol=tol)
            if r == 0:
                continue
            # Base indépendante via SVD
            U, S, Vh = np.linalg.svd(Vc, full_matrices=False)
            B = U[:, :r]  # colonnes indépendantes
            eigenvectors.append(B)
            diag_entries += [rep] * r
        else:
            eigenvectors.append(K)          # colonnes = base du noyau
            diag_entries += [rep] * k       # répéter rep k fois

    if len(eigenvectors) == 0:
        return False, None, None, "Aucun vecteur propre trouvé (numérique)."

    P = np.hstack(eigenvectors)
    # Réduire aux n premières colonnes linéairement indépendantes si surplus
    # (peut arriver si plusieurs bases se recouvrent)
    # Sélection par QR avec pivot
    Q, R = np.linalg.qr(P)
    # Rang par diagonale de R
    diag_R = np.abs(np.diag(R))
    independent = diag_R > tol * max(P.shape) * (diag_R[0] if diag_R.size and diag_R[0] != 0 else 1)
    rankP = int(np.sum(independent))

    if rankP < n:
        return False, None, None, "Matrice non diagonalisable (nombre de vecteurs propres indépendants < n)."

    # Tronquer P à n colonnes
    # On récupère les colonnes indépendantes via pivoting 'maison':
    # approche simple: recompute with pivoting using np.linalg.qr with mode 'reduced' and then reconstruct indices is non-trivial.
    # Plan B: sélection incrémentale
    cols = []
    selected = []
    for j in range(P.shape[1]):
        candidate = P[:, [j]] if len(cols) == 0 else np.hstack(cols + [P[:, [j]]])
        if matrix_rank(candidate, tol=tol) > len(cols):
            cols.append(P[:, [j]])
            selected.append(j)
        if len(cols) == n:
            break
    P = np.hstack(cols)
    # Construire D en référençant les lambdas associés aux colonnes sélectionnées
    # On reconstruit la liste diag_entries parallèle aux colonnes de P d'origine
    # Pour simplicité, on reclacule diag_entries sélectionnés dans le même ordre
    # On doit refaire l'association colonne->lambda ; comme nous avons empilé par blocs, on reconstruit
    lambdas_by_col = []
    offset = 0
    for rep, idxs in zip(reps, groups):
        # taille du bloc ajouté auparavant
        # soit K.shape[1] si noyau non vide, sinon 'r' ci-dessus (mais on ne l'a pas gardé)
        # Comme on a ajouté K.shape[1] colonnes à eigenvectors, reconstruisons en parcourant eigenvectors
        pass

    # Refaire plus simple : on recalcule les lambdas par proximité colonne par colonne
    # Chaque colonne p_j doit vérifier (A p_j) ~ lambda_j p_j ; on choisit lambda_j par least-squares
    # lambda_j = (p_j^* A p_j) / (p_j^* p_j) (Rayleigh quotient)
    diag_list = []
    for j in range(P.shape[1]):
        pj = P[:, j:j+1]
        num = (pj.conj().T @ (A @ pj))[0, 0]
        den = (pj.conj().T @ pj)[0, 0]
        lam_j = num / den
        diag_list.append(lam_j)
    # On ne garde que n colonnes (déjà tronqué)
    diag_list = diag_list[:n]
    D = np.diag(diag_list)

    if verbose:
        print("[Général] Diagonalisation calculée (si possible).")
        print("P =\n", P)
        print("D =\n", D)
        print("Vérif A P ~ P D :", np.allclose(A @ P, P @ D, atol=1e-7))

    return True, P, D, "OK" if np.allclose(A @ P, P @ D, atol=1e-7) else "Attention: précision numérique."

# -----------------------
# Démo
# -----------------------
if __name__ == "__main__":
    # Exemple diagonalisable non symétrique (3 valeurs propres distinctes)
    A1 = np.array([[4, 1, -2],
                   [0, 3,  1],
                   [0, 0,  2]], dtype=float)  # triangulaire -> diag = (4,3,2)
    ok, P, D, info = diagonalize(A1, verbose=True)
    print("Info A1:", info, "\n")

    # Exemple symétrique (toujours diagonalisable orthogonalement)
    A2 = np.array([[2, -1, 0],
                   [-1, 2, -1],
                   [0, -1, 2]], dtype=float)
    ok, P, D, info = diagonalize(A2, verbose=True)
    print("Info A2:", info, "\n")

    # Exemple non diagonalisable (bloc de Jordan 3x3)
    J = np.array([[2, 1, 0],
                  [0, 2, 1],
                  [0, 0, 2]], dtype=float)
    ok, P, D, info = diagonalize(J, verbose=True)
    print("Info J:", info)


[Général] Diagonalisation calculée (si possible).
P =
 [[ 1.      -0.j -0.707107-0.j -0.727607-0.j]
 [ 0.      -0.j  0.707107-0.j  0.485071-0.j]
 [ 0.      -0.j  0.      -0.j -0.485071-0.j]]
D =
 [[4.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 3.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 2.+0.j]]
Vérif A P ~ P D : True
Info A1: OK 

[Symétrique] Diagonalisation orthogonale trouvée.
P (orthonormale) =
 [[-0.5     +0.j -0.707107+0.j  0.5     +0.j]
 [-0.707107+0.j  0.      +0.j -0.707107+0.j]
 [-0.5     +0.j  0.707107+0.j  0.5     +0.j]]
D =
 [[0.585786+0.j 0.      +0.j 0.      +0.j]
 [0.      +0.j 2.      +0.j 0.      +0.j]
 [0.      +0.j 0.      +0.j 3.414214+0.j]]
Vérif A P ~ P D : True
Info A2: Symétrique réelle: diagonalisation orthogonale. 

Info J: Matrice non diagonalisable (nombre de vecteurs propres indépendants < n).
